## Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import csv
import datetime

In [2]:
sample = pd.read_csv("Sample Submission DAC.csv")
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

## Set column name as index and drop
train = train.set_index('Unnamed: 0')
train = train.reset_index(drop=True)
test = test.set_index('Unnamed: 0')
test = test.reset_index(drop=True)


In [3]:
## Set option to show column
pd.set_option('display.max_columns', None)


## Feature Selection 

In [2]:
train = pd.read_csv("train_clean.csv")
test = pd.read_csv('test_clean.csv')

In [3]:
print(train.shape)
print(test.shape)

(343583, 21)
(21018, 20)


In [4]:
test.drop(['Unnamed: 0'], axis = 1, inplace=True)


In [5]:
train.isnull().sum()

site                 0
continent_id         0
buyer_country        0
buyer_region         0
buyer_city           0
distance             0
buyer_id             0
mobile               0
package              0
channel_id           0
adults               0
children             0
room                 0
destination_id       0
destination_type     0
regency_continent    0
regency_country      0
regency_market       0
cnt                  0
regency_cluster      0
Number of days       0
dtype: int64

In [6]:
train.head()

,site,continent_id,buyer_country,buyer_region,buyer_city,distance,buyer_id,mobile,package,channel_id,...,children,room,destination_id,destination_type,regency_continent,regency_country,regency_market,cnt,regency_cluster,Number of days
0,2,3,66,348,48862,2234.2641,12,0,1,9,...,0,1,8250,1,3,2,50,628,1,4
1,2,3,66,348,48862,2234.2641,12,0,1,9,...,0,1,8250,1,1,2,50,628,1,4
2,2,3,66,348,48862,2234.2641,12,0,0,9,...,0,1,8250,1,1,2,50,628,1,4
3,2,3,66,442,35390,913.1932,93,0,0,3,...,0,1,14984,1,1,2,50,1457,80,5
4,2,3,66,442,35390,913.6259,93,0,0,3,...,0,1,14984,1,1,2,50,1457,21,5


In [15]:
RF_x_train = train.drop(['regency_cluster'], axis = 1)
RF_y_train = train['regency_cluster']

In [16]:
print(RF_x_train.shape)
print(RF_y_train.shape)

(343583, 20)
(343583,)


In [7]:
from sklearn.model_selection import train_test_split

## Split into train and test set
df_target = train['regency_cluster']
df_feature = train.drop(['regency_cluster', 'regency_country', 'continent_id', 'room', 'mobile', 'destination_type', 'package'], axis = 1)

In [8]:
print(df_target.shape)
print(df_feature.shape)

(343583,)
(343583, 14)


In [9]:
x_train, x_test, y_train, y_test = train_test_split(df_feature, df_target, test_size=0.2)


In [10]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(274866, 14)
(274866,)
(68717, 14)
(68717,)


## XGBoost 

In [22]:
!pip install xgboost

You should consider upgrading via the 'c:\users\denis\anaconda3\python.exe -m pip install --upgrade pip' command.


In [37]:
## model to use x_train and y_train
from xgboost import  XGBClassifier

model = XGBClassifier(n_estimator=50, max_depth=10, objective='binary:logistic')
model.fit(x_train, y_train)

C:\Users\denis\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:42:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:42:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimator=50, n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [38]:
feature_importances_1=pd.DataFrame({'features':df_feature.columns,'feature_importance':model.feature_importances_})
print(feature_importances_1.sort_values('feature_importance',ascending=False))


             features  feature_importance
16    regency_country            0.314326
17     regency_market            0.138453
18                cnt            0.074514
13     destination_id            0.067860
14   destination_type            0.036501
1        continent_id            0.031919
11           children            0.031189
2       buyer_country            0.031068
0                site            0.030254
6            buyer_id            0.028239
8             package            0.027653
3        buyer_region            0.025570
4          buyer_city            0.024239
10             adults            0.023617
19     Number of days            0.021977
9          channel_id            0.019904
7              mobile            0.019632
5            distance            0.019530
12               room            0.018956
15  regency_continent            0.014600


In [26]:
## model to use df_feature and train['regency_cluster']
from xgboost import  XGBClassifier 

model2=XGBClassifier()
model2.fit(df_feature,train['regency_cluster'])

[21:58:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [27]:
## Feature importance for XGB classifier
"""
Noted: 
        ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                        categorical type is supplied, DMatrix parameter
                        `enable_categorical` must be set to `True`.buying_date, dealing_date
                        
                        Therefore, buying_date & dealing_date will be drop
"""
 
feature_importances=pd.DataFrame({'features':df_feature.columns,'feature_importance':model2.feature_importances_})
print(feature_importances.sort_values('feature_importance',ascending=False))

             features  feature_importance
16    regency_country            0.299698
17     regency_market            0.142171
13     destination_id            0.078820
18                cnt            0.074474
11           children            0.049114
14   destination_type            0.043164
8             package            0.032818
3        buyer_region            0.031682
1        continent_id            0.028369
0                site            0.027300
2       buyer_country            0.024623
6            buyer_id            0.023959
10             adults            0.023129
19     Number of days            0.021510
5            distance            0.019472
4          buyer_city            0.019352
7              mobile            0.017171
9          channel_id            0.016219
12               room            0.016066
15  regency_continent            0.010888


## Random Forest


In [30]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=10, max_features=0.3202084571039361, min_samples_split=0.012864304237321418, n_estimators=555)
classifier.fit(RF_x_train, RF_y_train)

RandomForestClassifier(max_depth=10, max_features=0.3202084571039361,
                       min_samples_split=0.012864304237321418,
                       n_estimators=555)

In [32]:
## Feature importance for random forest classifier

RF_feature_importances = pd.DataFrame({'features':RF_x_train.columns,'feature_importance':classifier.feature_importances_})
print(RF_feature_importances.sort_values('feature_importance',ascending=False))

             features  feature_importance
18                cnt            0.271146
17     regency_market            0.228220
16    regency_country            0.215027
13     destination_id            0.137855
1        continent_id            0.024865
0                site            0.022444
8             package            0.017963
14   destination_type            0.016826
19     Number of days            0.015637
5            distance            0.015217
2       buyer_country            0.014461
3        buyer_region            0.008934
10             adults            0.006443
6            buyer_id            0.002476
11           children            0.001256
4          buyer_city            0.001102
12               room            0.000063
9          channel_id            0.000040
7              mobile            0.000022
15  regency_continent            0.000004


In [11]:
from sklearn.ensemble import RandomForestClassifier
#classifier2 = RandomForestClassifier(n_estimators = 10, random_state = 42)
classifier2 = RandomForestClassifier(max_depth=30, n_estimators=20, random_state=42)
classifier2.fit(x_train, y_train)

MemoryError: could not allocate 209715200 bytes

In [ ]:
feature_importances_3 = pd.DataFrame({'features':x_train.columns,'feature_importance':classifier2.feature_importances_})
print(feature_importances_3.sort_values('feature_importance',ascending=False))

## Evaluation

In [ ]:
print(classifier2.score(x_test,y_test))
print(classifier2.score(x_train,y_train))

In [46]:
y_pred_class = classifier2.predict(x_test)

In [47]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.11321798099451373


In [45]:
##evaluate Random Forest model 
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    test= (errors / test_labels)
    mape = 100 * (np.mean(errors)/np.mean(test_labels))
    print(mape)
    accuracy = 100 - mape
    print('Random Forest Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

RF_accuracy = evaluate(classifier2, x_test, y_test)

64.72960791628863
Random Forest Model Performance
Average Error: 32.0916 degrees.
Accuracy = 35.27%.


In [39]:
##evaluate XGBoost model 2
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    test= (errors / test_labels)
    mape = 100 * (np.mean(errors)/np.mean(test_labels))
    print(mape)
    accuracy = 100 - mape
    print('XGBoost Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

XGBoost_accuracy = evaluate(model2, x_test, y_test)

42.30313700431102
XGBoost Model Performance
Average Error: 20.9730 degrees.
Accuracy = 57.70%.


In [ ]:
##evaluate XGBoost model 1
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    test= (errors / test_labels)
    mape = 100 * (np.mean(errors)/np.mean(test_labels))
    print(mape)
    accuracy = 100 - mape
    print('XGBoost Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

XGBoost_accuracy = evaluate(model, x_test, y_test)

{'max_depth': 16,
 'max_features': 0.3202084571039361,
 'min_samples_split': 0.012864304237321418,
 'n_estimators': 555}

In [28]:
y_pred = classifier.predict(test)

IndexError: list index out of range

In [ ]:
y_pred

In [ ]:
#saving base model as a pickle file
import pickle
filename = 'RandomForestTuned.pkl'
pickle.dump(classifier, open(filename, 'wb'))

In [ ]:
def formatNumber(num):
    return int(num)
    

In [ ]:
#assuming id for test dataset is equal to the id of submission dataset
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv('Sample Submission DAC.csv')
datasets=pd.concat([sub_df['id'],pred],axis=1)
datasets.columns=['id','regency_cluster']
datasets.to_csv('sample_submission_test.csv',index=False)
final_sub_df = pd.read_csv('sample_submission_test.csv')
final_sub_df = final_sub_df.head(10001)
final_sub_df['id'] = final_sub_df['id'].astype(np.int64)
final_sub_df.to_csv('sample_submission_test.csv',index=False)

## Hyperparameter tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import pprint as pp
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import uniform, truncnorm, randint

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 40, num = 10)]



# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,       
               'max_features': truncnorm(a=0, b=1, loc=0.25, scale=0.1),
               'min_samples_split': uniform(0.01, 0.199)
               }
pp.pprint(random_grid)

# Random Search Training

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)


In [ ]:
# print winning set of hyperparameters
rf_random.best_params_